# Creating Post Nord dataset

### This was a project in which I collected data from different folders (including nested folders) into a single pandas dataframe. After that, I cleaned the data through, e.g. ensuring feature names were consistent across the different data sets so merging would happen correctly, reshaping values to fit same format, locating nulls and assigning them a value depending on different parameters etc. All in all the process was: merging datasets, finecombing the set for possible errors (or things that would prove problematic in the long run), and fixing said errors. Result was a cleaned data set that could be visualized in form of a dashboard.

In [2]:
import pandas as pd
import os

###  Loading the datasets to pandas dataframe
#### Loading and appending DDS sets, which is split up in four different folders, with up to 36 different excel files per folder. The end result is the DDS files concatenated to a single dataframe, df.

In [1]:
#DDS data
subdir_lst = []
files_lst = []
for subdir, dirs, files in os.walk(r'C:\Users\Emilp\OneDrive\Dokumenter\VA\POSTNORD\DDS'):
    subdir_lst.append(subdir)
    files_lst.append(files)

print(subdir_lst)
files_lst = [j for sub in files_lst for j in sub]

for sub in subdir_lst:
    for file in files_lst:
        try:
            cur_file = sub + '\\' + file
            xls_file = pd.read_excel(cur_file)
            df = df.append(xls_file, True)
        except:
            pass


NameError: name 'os' is not defined

In [1]:
#Exporting dataset to csv as to not load the data every time
#df = pd.read_csv(r'C:\Users\Emilp\dds_data.csv')
#df = df.drop(df.columns[0], axis=1)
#print(len(df))

#### Performing the same process on ADS, creating a different df which will later be merged with DDS df. ADS was simply split into three CSV files in the main folder. Same principle was applied, looping through the folder, loading each filename as a pandas file and appends it. Output is the three files appended to a single df.

In [20]:
#ADS data
df_ADS = pd.read_csv (r'C:\Users\Emilp\OneDrive\Dokumenter\VA\POSTNORD\ADS\2017.csv', delimiter = '|')
df_ADS = df_ADS.iloc[0:0]


subdir_lst = []
files_lst = []
for subdir, dirs, files in os.walk(r'C:\Users\Emilp\OneDrive\Dokumenter\VA\POSTNORD\ADS'):
    subdir_lst.append(subdir)
    files_lst.append(files)
print(files_lst)

print(subdir_lst)
files_lst = [j for sub in files_lst for j in sub]

for sub in subdir_lst:
    for file in files_lst:
        try:
            cur_file = sub + '\\' + file
            xls_file = pd.read_csv(cur_file, delimiter = '|')
            df_ADS = df_ADS.append(xls_file, True)
        except:
            pass



[['2017.csv', '2018.csv', '2019.csv', 'Description_of_Variables.docx', 'New Microsoft Excel Worksheet.xlsx']]
['C:\\Users\\Emilp\\OneDrive\\Dokumenter\\VA\\POSTNORD\\ADS']


#### Renaming IPI and inserting a Product column to ensure it is compatible with DDS set

In [21]:
#Renaming a column as the contents in this column fulfil the same job as a column in DDS.
df_ADS = df_ADS.rename(columns={"Involved Party ID": "Cust_ID/IPI"})

#Inserting a product column as DDS also has one, and I knew that the ADS data had a product type C.
df_ADS.insert(3,'Product','C')
df_ADS.head(1)

,Shipment Tracking Number,Cust_ID/IPI,ConsignorCountryCode,Product,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,ShipmentDate,Price_Paid,FileName
0,14384676,0,DANMARK,C,2000,DANMARK,1324.0,2017-05-03,87.962,2017


#### Changing country names to initials for compatibility

In [22]:
print(df_ADS["ConsigneeCountryCode"].value_counts())

df_ADS["ConsigneeCountryCode"].mask(df_ADS["ConsigneeCountryCode"].str.upper() == 'danmark'.upper(), 'DK', inplace=True)
df_ADS["ConsigneeCountryCode"].mask(df_ADS["ConsigneeCountryCode"].str.upper() == 'sweden'.upper(), 'SE', inplace=True)
df_ADS["ConsigneeCountryCode"].mask(df_ADS["ConsigneeCountryCode"].str.upper() == 'norway'.upper(), 'NO', inplace=True)

print(df_ADS["ConsigneeCountryCode"].value_counts())

df_ADS.head(1)


DANMARK    342673
SWEDEN       6091
Norway       5388
Sweden       4768
NORWAY       2208
sweDEN       1130
swedEN         98
Name: ConsigneeCountryCode, dtype: int64
DK    342673
SE     12087
NO      7596
Name: ConsigneeCountryCode, dtype: int64


,Shipment Tracking Number,Cust_ID/IPI,ConsignorCountryCode,Product,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,ShipmentDate,Price_Paid,FileName
0,14384676,0,DANMARK,C,2000,DK,1324.0,2017-05-03,87.962,2017


### Loading EasyForward data, changing data format and inserting product and filename columns

In [2]:
#Easy forward/roadfreight data
df_RF = pd.read_excel (r'C:\Users\Emilp\OneDrive\Dokumenter\VA\POSTNORD\EasyForward\Historical_Before_May_2019.xls')
print(df_RF.head(1))
#This dataset had the an individual feature revolving around the week, month and year in which a package was sent. 
#This was changed to fit a date format as to ensure compatability with the merging.
df_RF.insert(2,'Date',0)
df_RF['Week'] = df_RF['Year'].astype(str) + df_RF['Week'].astype(str)
df_RF['ShipmentDate'] = pd.to_datetime((df_RF['Week'].astype(int)-1).astype(str) + "1", format="%Y%U%w")
df_RF = df_RF.drop(columns=['Week','Month','Year'])

#Inserting a product type R because I knew this roadfreight data is referred to as type 'R'
#and a filename (which file it originated from)
df_RF.insert(3,'Product','R')
df_RF.insert(4,'FileName','Historical_Before_May_2019')

print(len(df_RF))
df_RF.head(1)

NameError: name 'pd' is not defined

### Importing master data
#### I also had accompanying master data for all three sets. DDS and ADS had common keys with the Cust_ID/IPI column in the master data, while the RF's key was the main customer. Thus, they would be split up in DDS+ADS and RF and merged later.

In [24]:
#Master data
df_MASTER_DDS = pd.read_excel(r'C:\Users\Emilp\OneDrive\Dokumenter\VA\POSTNORD\Master_Data\ADS_DDS_RF_CustomerNumbers_MasterData.xlsx', 'DDS')
df_MASTER_ADS = pd.read_excel(r'C:\Users\Emilp\OneDrive\Dokumenter\VA\POSTNORD\Master_Data\ADS_DDS_RF_CustomerNumbers_MasterData.xlsx', 'ADS')
df_MASTER_RF = pd.read_excel(r'C:\Users\Emilp\OneDrive\Dokumenter\VA\POSTNORD\Master_Data\ADS_DDS_RF_CustomerNumbers_MasterData.xlsx', 'Road Frieght')


In [25]:
#Same column names for DDS + ADS master.
df_MASTER_DDS = df_MASTER_DDS.rename(columns={"Customer ID": "Cust_ID/IPI"})
df_MASTER_ADS = df_MASTER_ADS.rename(columns={"Involved Party ID": "Cust_ID/IPI"})
df_MASTER_RF.insert(3, 'System','EasyForward')

df_MASTER = df_MASTER_DDS
df_MASTER = df_MASTER.append(df_MASTER_ADS)
print(len(df_MASTER))
print(df_MASTER["System"].value_counts())
df_MASTER.head(10)


5949
DDS    4229
ADS    1720
Name: System, dtype: int64


,Cust_ID/IPI,Main Customer,Industry Level 1,Industry Level 2,System,Allocated SAP Number
0,EEAAAACEJ,Customer 1,Retail,Retail/Sports & Leisure,DDS,270421113.0
1,JAAAAICCC,Customer 2,Retail,Retail/Department Store,DDS,242702055.0
2,HIAAAABDC,Customer 3,Retail,Retail/Sports & Leisure,DDS,242542422.0
3,FCCAAAEID,Customer 4,Transport,Freight,DDS,271264860.0
4,CGAAABDHE,Customer 5,Retail,Retail/Automotive,DDS,242044899.0
5,IJAAABEDG,Customer 6,Industry,Print,DDS,240199713.0
6,JEAAAAEBH,Customer 7,Wholesaler,Wholesale/Machinery,DDS,272040774.0
7,CGAAAABBI,Customer 8,Wholesaler,Wholesale/Other,DDS,240184734.0
8,IIAAAEIDJ,Customer 9,Retail,Retail/Automotive,DDS,241230030.0
9,HAAAAADBI,Customer 10,Retail,Retail/Sports & Leisure,DDS,240166038.0


### Gathering the three data sets (DDS, ADS and RF) in one set, df_tot

In [26]:
df_tot = df
df_tot = df_tot.rename(columns={"Customer_ID": "Cust_ID/IPI"})
df_tot.insert (1, 'Main Customer', 0)
df_tot = df_tot.append(df_ADS)
df_tot = df_tot.append(df_RF)
df_tot.head(1)

,Cust_ID/IPI,Main Customer,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Energy surcharge,Infrastructure surcharge,Multi-parcel surcharge,Private delivery surcharge,Road toll,Special goods surcharge,Price_Paid,Base_Price,FileName,Date
0,IAAAACFHE,0,DK,8210.0,NO,1178.0,10.0,G,NaN,DCCB5C8AOCZCD85D6,...,104.568,0.0,0.0,0.0,0.0,0.0,846.168,741.6,DatadumpProduct G_2017_apr,NaN


In [27]:
#Found out at this point I had overseen a mistake on a column, and ensured a uniform format.
df_tot["ConsignorCountryCode"].mask(df_tot["ConsignorCountryCode"].str.upper() == 'danmark'.upper(), 'DK', inplace=True)
df_tot['ConsignorCountryCode'].value_counts()

DK    3600465
Name: ConsignorCountryCode, dtype: int64

### Merges total data with the DDS/ADS master data on the customer ID / IPI column

In [28]:
df_tot_merged = df_tot.merge(df_MASTER, on='Cust_ID/IPI', how='left')
df_tot_merged.head(1)


,Cust_ID/IPI,Main Customer_x,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Special goods surcharge,Price_Paid,Base_Price,FileName,Date,Main Customer_y,Industry Level 1,Industry Level 2,System,Allocated SAP Number
0,IAAAACFHE,0,DK,8210.0,NO,1178.0,10.0,G,NaN,DCCB5C8AOCZCD85D6,...,0.0,846.168,741.6,DatadumpProduct G_2017_apr,NaN,Customer 2729,Retail,Retail/Media & Books,DDS,240220176.0


In [29]:
#The merge added copies of columns, some of which where the data were split. Thus, I would transfer all data
#from the copied column to the other, and delete the copy.
df_tot_merged["Main Customer_y"].mask(df_tot_merged["Main Customer_y"] == 0, df_tot_merged['Main Customer_x'], inplace=True)
df_tot_merged = df_tot_merged.rename(columns = {'Main Customer_y': 'Main Customer'})

print(df_tot_merged['Product'].value_counts())
df_tot_merged.head(1)

G    1405558
L    1072680
I     748193
C     362356
R      11678
Name: Product, dtype: int64


,Cust_ID/IPI,Main Customer_x,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Special goods surcharge,Price_Paid,Base_Price,FileName,Date,Main Customer,Industry Level 1,Industry Level 2,System,Allocated SAP Number
0,IAAAACFHE,0,DK,8210.0,NO,1178.0,10.0,G,NaN,DCCB5C8AOCZCD85D6,...,0.0,846.168,741.6,DatadumpProduct G_2017_apr,NaN,Customer 2729,Retail,Retail/Media & Books,DDS,240220176.0


### Creates a seperate dataset with only EasyForward data
#### Drops duplicates of Main Customer in master data for Easy Forward
#### Merges EasyForward and its Master data

In [32]:
mask = df_tot_merged['FileName'].values=='Historical_Before_May_2019'
df_R_only = df_tot_merged[mask]
del df_R_only['Main Customer']
df_R_only.head(1)



,Cust_ID/IPI,Main Customer_x,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Road toll,Special goods surcharge,Price_Paid,Base_Price,FileName,Date,Industry Level 1,Industry Level 2,System,Allocated SAP Number
3597225,NaN,Customer 3508,DK,NaN,NaN,NaN,NaN,R,NaN,NaN,...,NaN,NaN,6480.945,NaN,Historical_Before_May_2019,0.0,NaN,NaN,NaN,NaN


In [33]:
df_R_only = df_R_only.rename(columns={'Main Customer_x': 'Main Customer'})
df_R_only.head(1)

,Cust_ID/IPI,Main Customer,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Road toll,Special goods surcharge,Price_Paid,Base_Price,FileName,Date,Industry Level 1,Industry Level 2,System,Allocated SAP Number
3597225,NaN,Customer 3508,DK,NaN,NaN,NaN,NaN,R,NaN,NaN,...,NaN,NaN,6480.945,NaN,Historical_Before_May_2019,0.0,NaN,NaN,NaN,NaN


In [34]:
df_MASTER_RF = df_MASTER_RF.drop_duplicates('Main Customer')

df_R_merged = (df_R_only.merge(df_MASTER_RF,on= 'Main Customer', how='left'))

print(len(df_R_merged))
df_R_merged.head(1)

3240


,Cust_ID/IPI,Main Customer,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,FileName,Date,Industry Level 1_x,Industry Level 2_x,System_x,Allocated SAP Number_x,Industry Level 1_y,Industry Level 2_y,System_y,Allocated SAP Number_y
0,NaN,Customer 3508,DK,NaN,NaN,NaN,NaN,R,NaN,NaN,...,Historical_Before_May_2019,0.0,NaN,NaN,NaN,NaN,Transport,Freight,EasyForward,60957978


### Deletes columns from the merged EasyForward set that are not supposed to be there

In [36]:
del df_R_merged['Industry Level 1_x']
del df_R_merged['Industry Level 2_x']
del df_R_merged['System_x']
del df_R_merged['Allocated SAP Number_x']
df_R_merged = df_R_merged.rename(columns={"Industry Level 1_y": "Industry Level 1", "Industry Level 2_y": "Industry Level 2", "System_y":"System","Allocated SAP Number_y":"Allocated SAP Number"})

In [37]:
df_R_merged.head(1)

,Cust_ID/IPI,Main Customer,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Road toll,Special goods surcharge,Price_Paid,Base_Price,FileName,Date,Industry Level 1,Industry Level 2,System,Allocated SAP Number
0,NaN,Customer 3508,DK,NaN,NaN,NaN,NaN,R,NaN,NaN,...,NaN,NaN,6480.945,NaN,Historical_Before_May_2019,0.0,Transport,Freight,EasyForward,60957978


### Drops all EasyForward data from the total dataset

In [38]:
print(len(df_tot_merged))
print(df_tot_merged['Product'].value_counts())
df_tot_merged.drop(df_tot_merged[df_tot_merged.FileName == 'Historical_Before_May_2019'].index, inplace=True)
print(len(df_tot_merged))
print(df_tot_merged['Product'].value_counts())

3600465
G    1405558
L    1072680
I     748193
C     362356
R      11678
Name: Product, dtype: int64
3597225
G    1405558
L    1072680
I     748193
C     362356
R       8438
Name: Product, dtype: int64


### Appends the EasyForward data (including Master data) with the total dataset (including its master data)
#### Also looks at length and contents to ensure that nothing went wrong when merging with master data

In [57]:
df_total = df_tot_merged.append(df_R_merged)
print(len(df_total))
print(df_total['Product'].value_counts())

3600465
G    1405558
L    1072680
I     748193
C     362356
R      11678
Name: Product, dtype: int64


In [58]:
df_tot_merged.head(1)

,Cust_ID/IPI,Main Customer_x,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Special goods surcharge,Price_Paid,Base_Price,FileName,Date,Main Customer,Industry Level 1,Industry Level 2,System,Allocated SAP Number
0,IAAAACFHE,0,DK,8210.0,NO,1178.0,10.0,G,NaN,DCCB5C8AOCZCD85D6,...,0.0,846.168,741.6,DatadumpProduct G_2017_apr,NaN,Customer 2729,Retail,Retail/Media & Books,DDS,240220176.0


In [60]:
df_R_merged.head(1)

,Cust_ID/IPI,Main Customer,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Road toll,Special goods surcharge,Price_Paid,Base_Price,FileName,Date,Industry Level 1,Industry Level 2,System,Allocated SAP Number
0,NaN,Customer 3508,DK,NaN,NaN,NaN,NaN,R,NaN,NaN,...,NaN,NaN,6480.945,NaN,Historical_Before_May_2019,0.0,Transport,Freight,EasyForward,60957978


In [61]:
df_total = df_total.drop(['Date'], axis=1)

In [62]:
df_total.head(1)

,Cust_ID/IPI,Main Customer_x,ConsignorCountryCode,Consignor Location Nr,ConsigneeCountryCode,Consignee Location Nr,Freight calc_ status,Product,Parcel/Pallet type,Shipment Tracking Number,...,Road toll,Special goods surcharge,Price_Paid,Base_Price,FileName,Main Customer,Industry Level 1,Industry Level 2,System,Allocated SAP Number
0,IAAAACFHE,0,DK,8210.0,NO,1178.0,10.0,G,NaN,DCCB5C8AOCZCD85D6,...,0.0,0.0,846.168,741.6,DatadumpProduct G_2017_apr,Customer 2729,Retail,Retail/Media & Books,DDS,2.4022e+08


In [64]:
df_total['ShipmentDate'] = df_total.ShipmentDate.astype(str).str[0:11]

In [71]:
#Fills all rows missing indsutry level 1 with "Wholesaler"
df_total['Industry Level 1'].fillna('Wholesaler, inplace = True')

0            Retail
1          Industry
2          Industry
3        Wholesaler
4        Wholesaler
           ...     
3235    Missing SNI
3236    Missing SNI
3237    Missing SNI
3238    Missing SNI
3239      Transport
Name: Industry Level 1, Length: 3600465, dtype: object

## Dataset is now properly created and cleaned, thus it can be utilised to create a dashboard with
#### Exports the dataset to a csv file

In [73]:
df_total.to_csv('Postnord_cleaned.csv')